### Loading data

- PV/OP data
- SSD CSV
- Trip data

In [55]:
import pandas as pd

# Load OP PV data
df_related_KG_tags = pd.read_csv('DATA/03LIC1071_PropaneLoop_0426.csv')
# Get unique tag names from df_related_KG_tags
unique_tag_names = df_related_KG_tags['tagName'].unique()

op_pv_data_df = pd.read_parquet('/home/h604827/ControlActions/DATA/03LIC_1071_JAN_2026.parquet')
ssd_df = pd.read_excel("/home/h604827/ControlActions/DATA/SSD_1071_SSD_output_1071_7Jan2026.xlsx")

trip_data = pd.read_csv('/home/h604827/ControlActions/DATA/Final_List_Trip_Duration.csv')
op_pv_data_df.set_index('TimeStamp', inplace=True)
op_pv_data_df.sort_index(inplace=True)
op_pv_data_df.columns, trip_data.columns

(Index(['03LIC_1071.PV', '03LIC_1071.OP', '02FI_1000.PV', '03FIC_1085.OP',
        '03FIC_1085.PV', '03FIC_3415.OP', '03FIC_3415.PV', '03FIC_3435.PV',
        '03FI_1141A.PV', '03FI_1151.PV', '03FI_3418.PV', '03LIC_1016.OP',
        '03LIC_1016.PV', '03LIC_1085.OP', '03LIC_1085.PV', '03LIC_1094.OP',
        '03LIC_1094.PV', '03LIC_1097.OP', '03LIC_1097.PV', '03LIC_3178.OP',
        '03LIC_3178.PV', '03LI_3411.PV', '03PIC_1013.OP', '03PIC_1013.PV',
        '03PIC_1068.OP', '03PIC_1068.PV', '03PIC_1104.OP', '03PIC_1104.PV',
        '03PIC_3131.OP', '03PIC_3131.PV', '03PI_1141A.PV', '03PI_1495.PV',
        '03PI_1814.PV', '03TIC_1092.OP', '03TIC_1092.PV', '03TIC_1142.OP',
        '03TIC_1142.PV', '03TIC_1145.OP', '03TIC_1145.PV', '03TI_1015.PV',
        '03TI_1081.PV', '03TI_1421.PV', '03TI_1901.PV', 'AlarmStatus',
        'AlarmType'],
       dtype='object'),
 Index(['Stop Date', 'Start Date', 'Unit', 'SAP Location'], dtype='object'))

In [56]:
# Get the list of columns
cols = op_pv_data_df.columns

# Extract base tag names for OP and PV columns
op_tags = {col.replace('.OP', '') for col in cols if col.endswith('.OP')}
pv_tags = {col.replace('.PV', '') for col in cols if col.endswith('.PV')}

# Identify mismatches
op_without_pv = op_tags - pv_tags
pv_without_op = pv_tags - op_tags

print("Tags with OP but no PV:", op_without_pv)
print("Tags with PV but no OP:", pv_without_op)

Tags with OP but no PV: set()
Tags with PV but no OP: {'03TI_1901', '03TI_1081', '03PI_1141A', '03FI_3418', '03FI_1151', '03PI_1495', '03PI_1814', '03TI_1015', '03FIC_3435', '03TI_1421', '03FI_1141A', '02FI_1000', '03LI_3411'}


### Load events data and remove trip duration events

Filter out events that occur during plant trip/shutdown periods to focus on normal operating conditions.

In [57]:
combined_pv_events_df = pd.read_csv("/home/h604827/ControlActions/DATA/df_df_events_1071_export.csv", low_memory=False)
combined_pv_events_df['VT_Start'] = pd.to_datetime(combined_pv_events_df['VT_Start'])
combined_pv_events_df = combined_pv_events_df.sort_values('VT_Start')

# Convert trip_data dates to datetime
trip_data['Stop Date'] = pd.to_datetime(trip_data['Stop Date'])
trip_data['Start Date'] = pd.to_datetime(trip_data['Start Date'])

# Use numpy for fast vectorized comparison
import numpy as np

event_times = combined_pv_events_df['VT_Start'].values
stop_dates = trip_data['Stop Date'].values
start_dates = trip_data['Start Date'].values

# Check if each event falls within any trip period using broadcasting
# Shape: (n_events, n_trips) -> then reduce with any()
events_in_trips = np.zeros(len(event_times), dtype=bool)

# Process in chunks to avoid memory issues with large arrays
chunk_size = 100000
for i in range(0, len(event_times), chunk_size):
    chunk = event_times[i:i+chunk_size]
    # Broadcasting: (chunk_size, 1) compared with (1, n_trips)
    in_range = (chunk[:, None] >= stop_dates) & (chunk[:, None] <= start_dates)
    events_in_trips[i:i+chunk_size] = in_range.any(axis=1)

# Filter out events that are within trip periods
print(f"Original events: {len(combined_pv_events_df)}")
print(f"Events during trips (to be removed): {events_in_trips.sum()}")
combined_pv_events_df = combined_pv_events_df[~events_in_trips]
print(f"Remaining events after filtering: {len(combined_pv_events_df)}")

combined_pv_events_df.shape

Original events: 1947510
Events during trips (to be removed): 346990
Remaining events after filtering: 1600520


(1600520, 29)

In [58]:
# combined_pv_events_df.to_csv('/home/h604827/ControlActions/DATA/trip_filtered_events.csv', index=False)

In [59]:
# Remove trip duration data from op_pv_data_df

print("Filtering op_pv_data_df to remove trip periods...")
print(f"Original op_pv_data_df shape: {op_pv_data_df.shape}")

# Get timestamps from op_pv_data_df index
op_pv_timestamps = op_pv_data_df.index.values

# Use numpy for fast vectorized comparison
timestamps_in_trips = np.zeros(len(op_pv_timestamps), dtype=bool)

# Process in chunks to avoid memory issues with large arrays
chunk_size = 100000
for i in range(0, len(op_pv_timestamps), chunk_size):
    chunk = op_pv_timestamps[i:i+chunk_size]
    # Broadcasting: (chunk_size, 1) compared with (1, n_trips)
    in_range = (chunk[:, None] >= stop_dates) & (chunk[:, None] <= start_dates)
    timestamps_in_trips[i:i+chunk_size] = in_range.any(axis=1)

# Filter out timestamps that are within trip periods
print(f"Timestamps during trips (to be removed): {timestamps_in_trips.sum()}")
op_pv_data_df = op_pv_data_df[~timestamps_in_trips]
print(f"Remaining timestamps after filtering: {len(op_pv_data_df)}")
print(f"Filtered op_pv_data_df shape: {op_pv_data_df.shape}")

Filtering op_pv_data_df to remove trip periods...
Original op_pv_data_df shape: (1737586, 45)
Timestamps during trips (to be removed): 19547
Remaining timestamps after filtering: 1718039
Filtered op_pv_data_df shape: (1718039, 45)


In [60]:
# op_pv_data_df.to_parquet('/home/h604827/ControlActions/DATA/03LIC_1071_JAN_2026_filtered.parquet')

### Operator actions for PIC_1013, 1071 and 1016 tag

In [61]:
combined_pv_events_df[combined_pv_events_df['ConditionName'] == 'CHANGE']['Source'].value_counts()

Source
03LIC_1034      40978
03FIC_3435      34252
03HIC_1151      21948
03HIC_3100      19412
03PIC_1013      15568
                ...  
03RH_0201           2
03HS_0151           2
03LT_1071           2
03ST_3199A          2
03KI_0049_LB        2
Name: count, Length: 245, dtype: int64

In [62]:
combined_pv_events_df[(combined_pv_events_df['Source'].isin(['03LIC_1071', '03LIC_1016', '03PIC_1013'])) & (combined_pv_events_df['ConditionName'] == 'CHANGE')]['Source'].value_counts()

Source
03PIC_1013    15568
03LIC_1071     3076
03LIC_1016     1860
Name: count, dtype: int64

In [63]:
combined_pv_events_df[(combined_pv_events_df['Source'] =='03LIC_1016') & (combined_pv_events_df['ConditionName'] == 'CHANGE')]['Description'].value_counts()

Description
OP      1096
SP       642
MODE     122
Name: count, dtype: int64

### Operators changed the mode to MAN for 1071 and 1016 alarms these many times

In [64]:
combined_pv_events_df[(combined_pv_events_df['Source'].isin(['03LIC_1071', '03LIC_1016'])) & (combined_pv_events_df['ConditionName'] == 'CHANGE') & (combined_pv_events_df['Description'] == 'MODE') & (combined_pv_events_df['Value'] == 'MAN')]

,Action,Actor,AreaName,AlarmLimit,Block,Category,ConditionName,Description,EventID,Flags,...,SourceParameter,Station,Time,TransactionID,Units,Value,VT_Start,H,TagID,AlarmStatus
546921,NaN,NaN,1E,NaN,NaN,7,CHANGE,MODE,31971423,NaN,...,NaN,NaN,132822814613603000,31971423,%,MAN,2021-11-25 06:37:41.360300,2021_11_25_08,NaN,NaN
6663,NaN,NaN,1E,NaN,NaN,7,CHANGE,MODE,31971423,NaN,...,NaN,NaN,132822814613603000,31971423,%,MAN,2021-11-25 06:37:41.360300,2021_11_25_08,03LIC_1016,NaN
7519,NaN,NaN,1F,NaN,NaN,7,CHANGE,MODE,31971445,NaN,...,NaN,NaN,132822814848530000,31971445,%,MAN,2021-11-25 06:38:04.853000,2021_11_25_08,03LIC_1071,NaN
568524,NaN,NaN,1F,NaN,NaN,7,CHANGE,MODE,31971445,NaN,...,NaN,NaN,132822814848530000,31971445,%,MAN,2021-11-25 06:38:04.853000,2021_11_25_08,NaN,NaN
8511,NaN,NaN,1F,NaN,NaN,7,CHANGE,MODE,33209244,NaN,...,NaN,NaN,132834813552533000,33209244,%,MAN,2021-12-09 03:55:55.253300,2021_12_09_05,03LIC_1071,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
639544,NaN,NaN,1F,NaN,NaN,7,CHANGE,MODE,78594407,NaN,...,NaN,NaN,133911203810236000,78594407,%,MAN,2025-05-07 23:39:41.023600,2025_05_08_01,NaN,NaN
644160,NaN,NaN,1F,NaN,NaN,7,CHANGE,MODE,79410466,NaN,...,NaN,NaN,133949891544419000,79410466,%,MAN,2025-06-21 18:19:14.441900,2025_06_21_19,NaN,NaN
68375,NaN,NaN,1F,NaN,NaN,7,CHANGE,MODE,79410466,NaN,...,NaN,NaN,133949891544419000,79410466,%,MAN,2025-06-21 18:19:14.441900,2025_06_21_19,03LIC_1071,NaN
644171,NaN,NaN,1F,NaN,NaN,7,CHANGE,MODE,79410548,NaN,...,NaN,NaN,133949895420746000,79410548,%,MAN,2025-06-21 18:25:42.074600,2025_06_21_19,NaN,NaN


### Actions coming within [transition start, alarm end + 60 mins] for each alarm

In [65]:
# For each alarm episode, find the earliest Tag_First_Transition_Start_minutes
# and create a window from that timestamp to AlarmEnd + 60 minutes

import pandas as pd
import numpy as np

# Ensure datetime columns are properly parsed
ssd_df['Tag_First_Transition_Start_minutes'] = pd.to_datetime(ssd_df['Tag_First_Transition_Start_minutes'])
ssd_df['AlarmEnd_rounded_minutes'] = pd.to_datetime(ssd_df['AlarmEnd_rounded_minutes'])
ssd_df['AlarmStart_rounded_minutes'] = pd.to_datetime(ssd_df['AlarmStart_rounded_minutes'])

# Group by alarm episode (AlarmStart, AlarmEnd) and find the earliest transition start
alarm_windows = ssd_df.groupby(
    ['AlarmStart_rounded_minutes', 'AlarmEnd_rounded_minutes']
).agg(
    earliest_transition_start=('Tag_First_Transition_Start_minutes', 'min')
).reset_index()

# Create window end as AlarmEnd + 60 minutes
alarm_windows['window_start'] = alarm_windows['earliest_transition_start']
alarm_windows['window_end'] = alarm_windows['AlarmEnd_rounded_minutes'] + pd.Timedelta(minutes=60)

print(f"Total alarm windows: {len(alarm_windows)}")
print(f"\nFirst 5 alarm windows:")
print(alarm_windows[['AlarmStart_rounded_minutes', 'AlarmEnd_rounded_minutes', 
                      'window_start', 'window_end']].head())

# Now filter combined_pv_events_df for events within these windows
# Convert VT_Start to datetime if not already
combined_pv_events_df['VT_Start'] = pd.to_datetime(combined_pv_events_df['VT_Start'])

# Vectorized approach: check if each event falls within any window
event_times = combined_pv_events_df['VT_Start'].values
window_starts = alarm_windows['window_start'].values
window_ends = alarm_windows['window_end'].values

# Process in chunks to avoid memory issues
events_in_windows = np.zeros(len(event_times), dtype=bool)
alarm_episode_idx = np.full(len(event_times), -1, dtype=int)  # Track which alarm episode each event belongs to

chunk_size = 100000
for i in range(0, len(event_times), chunk_size):
    chunk = event_times[i:i+chunk_size]
    # Broadcasting: (chunk_size, 1) compared with (1, n_windows)
    in_range = (chunk[:, None] >= window_starts) & (chunk[:, None] <= window_ends)
    events_in_windows[i:i+chunk_size] = in_range.any(axis=1)
    # Get the first matching alarm episode index for each event
    for j, row_matches in enumerate(in_range):
        if row_matches.any():
            alarm_episode_idx[i+j] = np.where(row_matches)[0][0]

# Filter events that are within any alarm window
events_in_alarm_windows_df = combined_pv_events_df[events_in_windows].copy()
events_in_alarm_windows_df['AlarmEpisodeIndex'] = alarm_episode_idx[events_in_windows]

# Add alarm episode details
events_in_alarm_windows_df = events_in_alarm_windows_df.merge(
    alarm_windows[['AlarmStart_rounded_minutes', 'AlarmEnd_rounded_minutes', 
                   'window_start', 'window_end']].reset_index().rename(columns={'index': 'AlarmEpisodeIndex'}),
    on='AlarmEpisodeIndex',
    how='left'
)

events_in_alarm_windows_df = events_in_alarm_windows_df[(events_in_alarm_windows_df['Source'].isin(['03LIC_1071', '03LIC_1016', '03PIC_1013'])) & (events_in_alarm_windows_df['ConditionName'] == 'CHANGE')]
print(f"\n--- Events Filtering Summary ---")
print(f"Total events in combined_pv_events_df: {len(combined_pv_events_df)}")
print(f"Events within alarm windows: {len(events_in_alarm_windows_df)}")
print(f"Unique alarm episodes with events: {events_in_alarm_windows_df['AlarmEpisodeIndex'].nunique()}")
print(f"\nConditionName distribution in filtered events:")
print(events_in_alarm_windows_df['ConditionName'].value_counts().head(10))
print(f"\nSample filtered events:")
events_in_alarm_windows_df.head(10)

Total alarm windows: 609

First 5 alarm windows:
  AlarmStart_rounded_minutes AlarmEnd_rounded_minutes        window_start  \
0        2022-01-05 08:53:00      2022-01-05 09:33:00 2022-01-05 07:27:00   
1        2022-01-07 09:55:00      2022-01-07 10:00:00 2022-01-07 08:29:00   
2        2022-01-07 13:33:00      2022-01-07 13:36:00 2022-01-07 12:07:00   
3        2022-01-07 14:17:00      2022-01-07 14:19:00 2022-01-07 12:51:00   
4        2022-01-07 14:54:00      2022-01-07 14:58:00 2022-01-07 13:28:00   

           window_end  
0 2022-01-05 10:33:00  
1 2022-01-07 11:00:00  
2 2022-01-07 14:36:00  
3 2022-01-07 15:19:00  
4 2022-01-07 15:58:00  

--- Events Filtering Summary ---
Total events in combined_pv_events_df: 1600520
Events within alarm windows: 7958
Unique alarm episodes with events: 206

ConditionName distribution in filtered events:
ConditionName
CHANGE    7958
Name: count, dtype: int64

Sample filtered events:


,Action,Actor,AreaName,AlarmLimit,Block,Category,ConditionName,Description,EventID,Flags,...,Value,VT_Start,H,TagID,AlarmStatus,AlarmEpisodeIndex,AlarmStart_rounded_minutes,AlarmEnd_rounded_minutes,window_start,window_end
159,NaN,NaN,1L,NaN,NaN,7,CHANGE,MODE,35633314,NaN,...,MAN,2022-01-05 08:41:15.253100,2022_01_05_10,03PIC_1013,NaN,0,2022-01-05 08:53:00,2022-01-05 09:33:00,2022-01-05 07:27:00,2022-01-05 10:33:00
160,NaN,NaN,1L,NaN,NaN,7,CHANGE,MODE,35633314,NaN,...,MAN,2022-01-05 08:41:15.253100,2022_01_05_10,NaN,NaN,0,2022-01-05 08:53:00,2022-01-05 09:33:00,2022-01-05 07:27:00,2022-01-05 10:33:00
199,NaN,NaN,1E,NaN,NaN,7,CHANGE,SP,35633572,NaN,...,20.0000,2022-01-05 08:42:17.358300,2022_01_05_10,NaN,NaN,0,2022-01-05 08:53:00,2022-01-05 09:33:00,2022-01-05 07:27:00,2022-01-05 10:33:00
200,NaN,NaN,1E,NaN,NaN,7,CHANGE,SP,35633572,NaN,...,20.0000,2022-01-05 08:42:17.358300,2022_01_05_10,03LIC_1016,NaN,0,2022-01-05 08:53:00,2022-01-05 09:33:00,2022-01-05 07:27:00,2022-01-05 10:33:00
207,NaN,NaN,1L,NaN,NaN,7,CHANGE,OP,35633602,NaN,...,78.3480,2022-01-05 08:42:23.715200,2022_01_05_10,03PIC_1013,NaN,0,2022-01-05 08:53:00,2022-01-05 09:33:00,2022-01-05 07:27:00,2022-01-05 10:33:00
208,NaN,NaN,1L,NaN,NaN,7,CHANGE,OP,35633602,NaN,...,78.3480,2022-01-05 08:42:23.715200,2022_01_05_10,NaN,NaN,0,2022-01-05 08:53:00,2022-01-05 09:33:00,2022-01-05 07:27:00,2022-01-05 10:33:00
210,NaN,NaN,1L,NaN,NaN,7,CHANGE,OP,35633612,NaN,...,76.3480,2022-01-05 08:42:24.781600,2022_01_05_10,NaN,NaN,0,2022-01-05 08:53:00,2022-01-05 09:33:00,2022-01-05 07:27:00,2022-01-05 10:33:00
212,NaN,NaN,1L,NaN,NaN,7,CHANGE,OP,35633612,NaN,...,76.3480,2022-01-05 08:42:24.781600,2022_01_05_10,03PIC_1013,NaN,0,2022-01-05 08:53:00,2022-01-05 09:33:00,2022-01-05 07:27:00,2022-01-05 10:33:00
213,NaN,NaN,1F,NaN,NaN,7,CHANGE,SP,35633626,NaN,...,20.0000,2022-01-05 08:42:28.203100,2022_01_05_10,NaN,NaN,0,2022-01-05 08:53:00,2022-01-05 09:33:00,2022-01-05 07:27:00,2022-01-05 10:33:00
214,NaN,NaN,1F,NaN,NaN,7,CHANGE,SP,35633626,NaN,...,20.0000,2022-01-05 08:42:28.203100,2022_01_05_10,03LIC_1071,NaN,0,2022-01-05 08:53:00,2022-01-05 09:33:00,2022-01-05 07:27:00,2022-01-05 10:33:00


### Analysis: Episodes WITHOUT any actions on target tags (1071, 1016, 1013)

We want to find alarm episodes where **NONE** of the operator actions (CHANGE events) during the action window are on the three target tags:
- `03LIC_1071` - Target level controller
- `03LIC_1016` - Related level controller  
- `03PIC_1013` - Pressure controller (highly operated)

This helps us analyze episodes where operators took actions on OTHER tags to resolve the alarm.

In [66]:
# Step 1: Get all CHANGE events within alarm windows (using existing alarm_windows)
# Tags we are interested in EXCLUDING (target tags)
TARGET_TAGS = ['03LIC_1071', '03LIC_1016', '03PIC_1013']

# Get all CHANGE events
all_change_events = combined_pv_events_df[combined_pv_events_df['ConditionName'] == 'CHANGE'].copy()

print(f"Total CHANGE events: {len(all_change_events)}")
print(f"Unique sources with CHANGE events: {all_change_events['Source'].nunique()}")

# Find events within each alarm window
event_times = all_change_events['VT_Start'].values
window_starts_arr = alarm_windows['window_start'].values
window_ends_arr = alarm_windows['window_end'].values

# Track which episode each event belongs to
event_episode_idx = np.full(len(event_times), -1, dtype=int)

chunk_size = 100000
for i in range(0, len(event_times), chunk_size):
    chunk = event_times[i:i+chunk_size]
    in_range = (chunk[:, None] >= window_starts_arr) & (chunk[:, None] <= window_ends_arr)
    for j, row_matches in enumerate(in_range):
        if row_matches.any():
            event_episode_idx[i+j] = np.where(row_matches)[0][0]

# Add episode index to events
all_change_events['EpisodeIndex'] = event_episode_idx

# Filter to events within windows
change_events_in_windows = all_change_events[all_change_events['EpisodeIndex'] >= 0].copy()
print(f"\nCHANGE events within alarm windows: {len(change_events_in_windows)}")
print(f"Unique episodes with any CHANGE events: {change_events_in_windows['EpisodeIndex'].nunique()}")

Total CHANGE events: 231714
Unique sources with CHANGE events: 245

CHANGE events within alarm windows: 34908
Unique episodes with any CHANGE events: 488


In [67]:
# Step 2: For each episode, find which have NO actions on target tags (1071, 1016, 1013)
# An episode qualifies if:
# 1. It has at least one CHANGE event (some operator action)
# 2. NONE of the CHANGE events in that episode are on target tags

# Group by episode and get all unique sources (tags) that were operated on
episode_sources = change_events_in_windows.groupby('EpisodeIndex')['Source'].apply(set).reset_index()
episode_sources.columns = ['EpisodeIndex', 'OperatedTags']

# Check if an episode has NO target tag action
episode_sources['has_no_target_tag_action'] = episode_sources['OperatedTags'].apply(
    lambda tags: len(tags.intersection(set(TARGET_TAGS))) == 0
)

# Episodes with NO target tag actions (these are the ones we want)
episodes_no_target_tags = episode_sources[episode_sources['has_no_target_tag_action']]

# Episodes that HAVE target tag actions (for comparison)
episodes_with_target_tags = episode_sources[~episode_sources['has_no_target_tag_action']]

print(f"--- Episode Analysis ---")
print(f"Total unique alarm episodes (from SSD): {len(alarm_windows)}")
print(f"Episodes with any CHANGE events: {len(episode_sources)}")
print(f"Episodes with target tag actions (EXCLUDED): {len(episodes_with_target_tags)}")
print(f"Episodes with NO target tag actions (INCLUDED): {len(episodes_no_target_tags)}")

# Show breakdown of which tags were operated in the included episodes
print(f"\n--- Operated Tags in Included Episodes (no target tags) ---")
all_operated_tags_flat = []
for tags in episodes_no_target_tags['OperatedTags']:
    all_operated_tags_flat.extend(list(tags))
operated_tag_counts = pd.Series(all_operated_tags_flat).value_counts()
print(f"Total unique operated tags: {len(operated_tag_counts)}")
print(f"\nTop 20 most frequently operated tags:")
print(operated_tag_counts.head(20))

--- Episode Analysis ---
Total unique alarm episodes (from SSD): 609
Episodes with any CHANGE events: 488
Episodes with target tag actions (EXCLUDED): 206
Episodes with NO target tag actions (INCLUDED): 282

--- Operated Tags in Included Episodes (no target tags) ---
Total unique operated tags: 89

Top 20 most frequently operated tags:
03FIC_3435       191
03HIC_1151        95
03LIC_1034        84
03HIC_3100        69
03HIC_3132        30
03HIC_1141        28
03LIC_1085        26
03FIC_3415        25
03TIC_1009        18
03PIC_3131         9
03LIC_3408         8
03LIC_3153         8
04RES_HYP1COM      7
03PIC_1068         7
03FIC_1085         7
03SDV_1167         7
03LIC_1097         7
03HIC_1092A        6
03GM_0153C         5
03GM_0151A         4
Name: count, dtype: int64


In [68]:
# Step 3: Get details of the filtered episodes (those with NO target tag actions)

filtered_episode_indices = episodes_no_target_tags['EpisodeIndex'].values

# Create episodes dataframe with all details
filtered_episodes = alarm_windows.iloc[filtered_episode_indices].copy()
filtered_episodes['EpisodeIndex'] = filtered_episode_indices
filtered_episodes['EpisodeID'] = range(1, len(filtered_episodes) + 1)

# Rename columns to match expected format
filtered_episodes = filtered_episodes.rename(columns={
    'earliest_transition_start': 'EarliestTransitionStart'
})

# Calculate durations
filtered_episodes['AlarmDurationMinutes'] = (
    (filtered_episodes['AlarmEnd_rounded_minutes'] - filtered_episodes['AlarmStart_rounded_minutes'])
    .dt.total_seconds() / 60
)
filtered_episodes['TransitionToAlarmMinutes'] = (
    (filtered_episodes['AlarmStart_rounded_minutes'] - filtered_episodes['EarliestTransitionStart'])
    .dt.total_seconds() / 60
)
filtered_episodes['TotalWindowMinutes'] = (
    (filtered_episodes['window_end'] - filtered_episodes['window_start'])
    .dt.total_seconds() / 60
)

# Add info about which tags were operated in each episode
filtered_episodes = filtered_episodes.merge(
    episodes_no_target_tags[['EpisodeIndex', 'OperatedTags']],
    on='EpisodeIndex',
    how='left'
)
filtered_episodes['OperatedTagsList'] = filtered_episodes['OperatedTags'].apply(lambda x: ', '.join(sorted(x)))

print(f"--- Filtered Episodes Summary (NO target tag actions) ---")
print(f"Total episodes to analyze: {len(filtered_episodes)}")
print(f"\nDuration Statistics (minutes):")
print(f"  Alarm Duration - Mean: {filtered_episodes['AlarmDurationMinutes'].mean():.1f}, "
      f"Median: {filtered_episodes['AlarmDurationMinutes'].median():.1f}, "
      f"Min: {filtered_episodes['AlarmDurationMinutes'].min():.1f}, "
      f"Max: {filtered_episodes['AlarmDurationMinutes'].max():.1f}")
print(f"  Transition to Alarm - Mean: {filtered_episodes['TransitionToAlarmMinutes'].mean():.1f}, "
      f"Median: {filtered_episodes['TransitionToAlarmMinutes'].median():.1f}")
print(f"\nDate Range:")
print(f"  First episode: {filtered_episodes['AlarmStart_rounded_minutes'].min()}")
print(f"  Last episode: {filtered_episodes['AlarmStart_rounded_minutes'].max()}")

# Show first few episodes
print(f"\nFirst 10 filtered episodes:")
filtered_episodes[['EpisodeID', 'AlarmStart_rounded_minutes', 'AlarmEnd_rounded_minutes', 
                   'AlarmDurationMinutes', 'OperatedTagsList']].head(10)

--- Filtered Episodes Summary (NO target tag actions) ---
Total episodes to analyze: 282

Duration Statistics (minutes):
  Alarm Duration - Mean: 18.3, Median: 6.0, Min: 0.0, Max: 356.0
  Transition to Alarm - Mean: 84.2, Median: 86.0

Date Range:
  First episode: 2022-01-07 09:55:00
  Last episode: 2025-05-22 16:33:00

First 10 filtered episodes:


,EpisodeID,AlarmStart_rounded_minutes,AlarmEnd_rounded_minutes,AlarmDurationMinutes,OperatedTagsList
0,1,2022-01-07 09:55:00,2022-01-07 10:00:00,5.0,"03FIC_3435, 03GHS_0121A, 03GHS_0121AA, 03GHS_0..."
1,2,2022-01-07 13:33:00,2022-01-07 13:36:00,3.0,"03FIC_3435, 03LIC_1034"
2,3,2022-01-07 14:54:00,2022-01-07 14:58:00,4.0,03FIC_3435
3,4,2022-01-07 19:56:00,2022-01-07 20:02:00,6.0,03FIC_3435
4,5,2022-01-07 23:28:00,2022-01-07 23:31:00,3.0,03FIC_3435
5,6,2022-01-10 10:09:00,2022-01-10 10:15:00,6.0,03FIC_3435
6,7,2022-01-12 10:41:00,2022-01-12 10:45:00,4.0,03FIC_3435
7,8,2022-01-14 11:28:00,2022-01-14 11:35:00,7.0,03FIC_3435
8,9,2022-01-16 04:17:00,2022-01-16 04:24:00,7.0,03FIC_3435
9,10,2022-01-16 16:39:00,2022-01-16 16:45:00,6.0,"03FIC_3435, 03LIC_1034"


In [69]:
# Step 3b: Detailed statistics on the filtered episodes (NO target tag actions)

print("=" * 70)
print("DETAILED STATISTICS: Episodes WITHOUT Target Tag Actions (1071/1016/1013)")
print("=" * 70)

# Overall counts
print(f"\n📊 OVERALL COUNTS")
print(f"   Total SSD alarm episodes: {len(alarm_windows)}")
print(f"   Episodes with any CHANGE events: {len(episode_sources)}")
print(f"   Episodes WITH target tag actions (EXCLUDED): {len(episodes_with_target_tags)}")
print(f"   Episodes WITHOUT target tag actions (INCLUDED): {len(filtered_episodes)} ({100*len(filtered_episodes)/len(alarm_windows):.1f}%)")

# Tag analysis
print(f"\n📊 OPERATED TAGS IN INCLUDED EPISODES")
tag_freq = pd.Series(all_operated_tags_flat).value_counts()
print(f"   Unique operated tags: {len(tag_freq)}")
print(f"\n   Top 15 most frequently operated tags:")
for tag, count in tag_freq.head(15).items():
    print(f"   {count:4d} times - {tag}")

# Actions per episode
print(f"\n📊 ACTIONS PER EPISODE")
actions_per_episode_filtered = change_events_in_windows[
    change_events_in_windows['EpisodeIndex'].isin(filtered_episode_indices)
].groupby('EpisodeIndex').size()
print(f"   Mean actions per episode: {actions_per_episode_filtered.mean():.1f}")
print(f"   Median actions per episode: {actions_per_episode_filtered.median():.1f}")
print(f"   Min: {actions_per_episode_filtered.min()}, Max: {actions_per_episode_filtered.max()}")

# Duration analysis
print(f"\n📊 ALARM DURATION ANALYSIS (filtered episodes)")
print(f"   Mean: {filtered_episodes['AlarmDurationMinutes'].mean():.1f} min")
print(f"   Median: {filtered_episodes['AlarmDurationMinutes'].median():.1f} min")
print(f"   Std Dev: {filtered_episodes['AlarmDurationMinutes'].std():.1f} min")
print(f"   Min: {filtered_episodes['AlarmDurationMinutes'].min():.1f} min")
print(f"   Max: {filtered_episodes['AlarmDurationMinutes'].max():.1f} min")

# Time distribution
print(f"\n📊 TIME DISTRIBUTION BY YEAR")
filtered_episodes['Year'] = filtered_episodes['AlarmStart_rounded_minutes'].dt.year
year_dist = filtered_episodes.groupby('Year').agg({
    'EpisodeID': 'count',
    'AlarmDurationMinutes': 'mean'
}).rename(columns={'EpisodeID': 'Count', 'AlarmDurationMinutes': 'AvgDuration'})
print(year_dist.to_string())

DETAILED STATISTICS: Episodes WITHOUT Target Tag Actions (1071/1016/1013)

📊 OVERALL COUNTS
   Total SSD alarm episodes: 609
   Episodes with any CHANGE events: 488
   Episodes WITH target tag actions (EXCLUDED): 206
   Episodes WITHOUT target tag actions (INCLUDED): 282 (46.3%)

📊 OPERATED TAGS IN INCLUDED EPISODES
   Unique operated tags: 89

   Top 15 most frequently operated tags:
    191 times - 03FIC_3435
     95 times - 03HIC_1151
     84 times - 03LIC_1034
     69 times - 03HIC_3100
     30 times - 03HIC_3132
     28 times - 03HIC_1141
     26 times - 03LIC_1085
     25 times - 03FIC_3415
     18 times - 03TIC_1009
      9 times - 03PIC_3131
      8 times - 03LIC_3408
      8 times - 03LIC_3153
      7 times - 04RES_HYP1COM
      7 times - 03PIC_1068
      7 times - 03FIC_1085

📊 ACTIONS PER EPISODE
   Mean actions per episode: 42.1
   Median actions per episode: 18.0
   Min: 2, Max: 534

📊 ALARM DURATION ANALYSIS (filtered episodes)
   Mean: 18.3 min
   Median: 6.0 min
   Std 

In [70]:
# Step 4: Create visualization function with window timestamps in title
import importlib.util
from pathlib import Path

# Load generate_episode_plots module directly from script path
_gep_path = Path('.skills/episode-analyzer/scripts/generate_episode_plots.py')
_spec = importlib.util.spec_from_file_location('generate_episode_plots', _gep_path)
_gep = importlib.util.module_from_spec(_spec)
_spec.loader.exec_module(_gep)

# Read config-driven operator action settings
_action_mode = _gep.CONFIG.get('visualization', {}).get('operator_action_mode', 'list')
_max_tags = _gep.CONFIG.get('visualization', {}).get('max_operator_action_tags', 20)

# Load operating limits (used by generate_episode_plots)
limits_df = _gep.load_operating_limits('DATA/operating_limits.csv')

def create_episode_visualization_with_timestamps(episode, ts_df, events_df, context_minutes=30):
    """
    Create comprehensive visualization for a single episode using the same
    template as generate_episode_plots.py (all tags + OP/PV grouping).
    
    Modified to include window start/end timestamps in the title.
    """
    fig = _gep.create_episode_plot(
        episode=episode,
        ts_df=ts_df,
        events_df=events_df,
        ssd_df=ssd_df,
        limits_df=limits_df,
        context_minutes=context_minutes
    )
    
    # Update title to include window start and end timestamps
    episode_id = episode['EpisodeID']
    window_start = episode['window_start']
    window_end = episode['window_end']
    alarm_start = episode['AlarmStart_rounded_minutes']
    alarm_end = episode['AlarmEnd_rounded_minutes']
    alarm_duration = (alarm_end - alarm_start).total_seconds() / 60
    
    fig.update_layout(
        title=dict(
            text=(
                f'Episode {episode_id}: Alarm from {alarm_start.strftime("%Y-%m-%d %H:%M")} to {alarm_end.strftime("%H:%M")}<br>'
                f'<sup>Window: {window_start.strftime("%Y-%m-%d %H:%M")} to {window_end.strftime("%Y-%m-%d %H:%M")} | '
                f'Duration: {alarm_duration:.0f} min | No actions on target tags (1071/1016/1013)</sup>'
            ),
            x=0.5
        )
    )
    
    return fig

print("Visualization function created successfully!")
print(f"Operator action mode: {_action_mode} | Max tags: {_max_tags}")

Visualization function created successfully!
Operator action mode: all | Max tags: 200


### Export Episode Data to Excel and Generate Plots

Create an Excel file with:
- One row per (episode, operated tag) combination
- Columns for action counts: SP, OP, MODE

Also save all episode visualizations to `RESULTS/episode_no_target_tags_plots/`

Only considering episodes where **NO** target tags (1071, 1016, 1013) were operated.

In [71]:
# Step 5: Create Excel file with episode details - one row per (episode, operated tag)
# Columns: EpisodeID, Tag, SP_count, OP_count, MODE_count, TotalActions, OtherActionTypes
import os
from pathlib import Path

# Create output directory for plots and excel
output_dir = Path('RESULTS/episode_no_target_tags_plots')
output_dir.mkdir(parents=True, exist_ok=True)

# Get all CHANGE events for filtered episodes (those with NO target tag actions)
filtered_change_events = change_events_in_windows[
    change_events_in_windows['EpisodeIndex'].isin(filtered_episode_indices)
].copy()

# Map EpisodeIndex to EpisodeID
episode_idx_to_id = dict(zip(filtered_episodes['EpisodeIndex'], filtered_episodes['EpisodeID']))
filtered_change_events['EpisodeID'] = filtered_change_events['EpisodeIndex'].map(episode_idx_to_id)

# Group by EpisodeID, Source (tag), and Description (action type: SP, OP, MODE)
# Count occurrences of each action type
action_counts = filtered_change_events.groupby(
    ['EpisodeID', 'Source', 'Description']
).size().reset_index(name='Count')

print(f"Action types found in data:")
print(action_counts['Description'].value_counts())

# Identify known action types and "other" action types
KNOWN_ACTION_TYPES = ['SP', 'OP', 'MODE']
all_action_types = action_counts['Description'].unique()
other_action_types = [t for t in all_action_types if t not in KNOWN_ACTION_TYPES]
print(f"\nKnown action types: {KNOWN_ACTION_TYPES}")
print(f"Other action types found: {other_action_types}")

# Pivot to get SP, OP, MODE as columns
action_pivot = action_counts.pivot_table(
    index=['EpisodeID', 'Source'],
    columns='Description',
    values='Count',
    fill_value=0
).reset_index()

# Flatten column names
action_pivot.columns.name = None

# Ensure SP, OP, MODE columns exist (even if no such actions exist)
for col in ['SP', 'OP', 'MODE']:
    if col not in action_pivot.columns:
        action_pivot[col] = 0

# Calculate TotalActions (sum of all action types, including SP, OP, MODE and others)
action_type_cols = [c for c in action_pivot.columns if c not in ['EpisodeID', 'Source']]
action_pivot['TotalActions'] = action_pivot[action_type_cols].sum(axis=1)

# Calculate OtherActionTypes - list of action types that are not SP, OP, MODE
other_cols = [c for c in action_type_cols if c not in KNOWN_ACTION_TYPES]
if other_cols:
    # For each row, find which "other" columns have non-zero counts and list them
    def get_other_types(row):
        types_with_counts = []
        for col in other_cols:
            if row[col] > 0:
                types_with_counts.append(f"{col}({int(row[col])})")
        return ', '.join(types_with_counts) if types_with_counts else ''
    action_pivot['OtherActionTypes'] = action_pivot.apply(get_other_types, axis=1)
else:
    action_pivot['OtherActionTypes'] = ''

# Rename 'Source' to 'OperatedTag' for clarity
action_pivot = action_pivot.rename(columns={'Source': 'OperatedTag'})

# Add episode metadata (timestamps, duration)
episode_meta = filtered_episodes[['EpisodeID', 'AlarmStart_rounded_minutes', 'AlarmEnd_rounded_minutes', 
                                   'AlarmDurationMinutes', 'window_start', 'window_end']].copy()
episode_meta = episode_meta.rename(columns={
    'AlarmStart_rounded_minutes': 'AlarmStart',
    'AlarmEnd_rounded_minutes': 'AlarmEnd',
    'window_start': 'WindowStart',
    'window_end': 'WindowEnd'
})

# Merge episode metadata with action data
episode_tag_actions = action_pivot.merge(episode_meta, on='EpisodeID', how='left')

# Reorder columns - include TotalActions and OtherActionTypes
column_order = ['EpisodeID', 'OperatedTag', 'SP', 'OP', 'MODE', 'TotalActions', 'OtherActionTypes',
                'AlarmStart', 'AlarmEnd', 'AlarmDurationMinutes', 'WindowStart', 'WindowEnd']
# Only include columns that exist
column_order = [c for c in column_order if c in episode_tag_actions.columns]
episode_tag_actions = episode_tag_actions[column_order]

# Sort by EpisodeID and OperatedTag
episode_tag_actions = episode_tag_actions.sort_values(['EpisodeID', 'OperatedTag'])

# Save to Excel
excel_path = output_dir / 'episodes_no_target_tags_actions.xlsx'
episode_tag_actions.to_excel(excel_path, index=False)

print(f"\n✅ Saved episode data to: {excel_path}")
print(f"   Total rows (episode-tag combinations): {len(episode_tag_actions)}")
print(f"   Unique episodes: {episode_tag_actions['EpisodeID'].nunique()}")
print(f"   Unique operated tags: {episode_tag_actions['OperatedTag'].nunique()}")

print(f"\nAction type summary:")
for col in ['SP', 'OP', 'MODE']:
    if col in episode_tag_actions.columns:
        total = episode_tag_actions[col].sum()
        non_zero = (episode_tag_actions[col] > 0).sum()
        print(f"   {col}: {total} total actions across {non_zero} (episode,tag) pairs")

print(f"\nTotal actions summary:")
print(f"   Sum of all actions: {episode_tag_actions['TotalActions'].sum()}")
print(f"   Rows with other action types: {(episode_tag_actions['OtherActionTypes'] != '').sum()}")

if other_action_types:
    print(f"\nOther action types breakdown:")
    for ot in other_action_types:
        if ot in action_pivot.columns:
            total = action_pivot[ot].sum()
            print(f"   {ot}: {total} total actions")

print(f"\nSample rows:")
episode_tag_actions.head(15)

Action types found in data:
Description
OP          570
SP          177
MODE        151
PVFL         14
SO           11
PVSOURCE      1
TF            1
Name: count, dtype: int64

Known action types: ['SP', 'OP', 'MODE']
Other action types found: ['PVFL', 'SO', 'PVSOURCE', 'TF']

✅ Saved episode data to: RESULTS/episode_no_target_tags_plots/episodes_no_target_tags_actions.xlsx
   Total rows (episode-tag combinations): 759
   Unique episodes: 282
   Unique operated tags: 89

Action type summary:
   SP: 2064.0 total actions across 177 (episode,tag) pairs
   OP: 9144.0 total actions across 570 (episode,tag) pairs
   MODE: 574.0 total actions across 151 (episode,tag) pairs

Total actions summary:
   Sum of all actions: 11882.0
   Rows with other action types: 27

Other action types breakdown:
   PVFL: 30.0 total actions
   SO: 66.0 total actions
   PVSOURCE: 2.0 total actions
   TF: 2.0 total actions

Sample rows:


,EpisodeID,OperatedTag,SP,OP,MODE,TotalActions,OtherActionTypes,AlarmStart,AlarmEnd,AlarmDurationMinutes,WindowStart,WindowEnd
0,1,03FIC_3435,0.0,8.0,0.0,8.0,,2022-01-07 09:55:00,2022-01-07 10:00:00,5.0,2022-01-07 08:29:00,2022-01-07 11:00:00
1,1,03GHS_0121A,0.0,0.0,0.0,2.0,PVFL(2),2022-01-07 09:55:00,2022-01-07 10:00:00,5.0,2022-01-07 08:29:00,2022-01-07 11:00:00
2,1,03GHS_0121AA,0.0,0.0,0.0,2.0,PVFL(2),2022-01-07 09:55:00,2022-01-07 10:00:00,5.0,2022-01-07 08:29:00,2022-01-07 11:00:00
3,1,03GHS_0121B,0.0,0.0,0.0,2.0,PVFL(2),2022-01-07 09:55:00,2022-01-07 10:00:00,5.0,2022-01-07 08:29:00,2022-01-07 11:00:00
4,1,03GM_0121,0.0,2.0,0.0,2.0,,2022-01-07 09:55:00,2022-01-07 10:00:00,5.0,2022-01-07 08:29:00,2022-01-07 11:00:00
5,1,03GM_0121A,0.0,2.0,0.0,2.0,,2022-01-07 09:55:00,2022-01-07 10:00:00,5.0,2022-01-07 08:29:00,2022-01-07 11:00:00
6,1,03HIC_3100,0.0,4.0,0.0,4.0,,2022-01-07 09:55:00,2022-01-07 10:00:00,5.0,2022-01-07 08:29:00,2022-01-07 11:00:00
7,1,04RES_3ER1AD,0.0,26.0,0.0,52.0,SO(26),2022-01-07 09:55:00,2022-01-07 10:00:00,5.0,2022-01-07 08:29:00,2022-01-07 11:00:00
8,2,03FIC_3435,0.0,6.0,0.0,6.0,,2022-01-07 13:33:00,2022-01-07 13:36:00,3.0,2022-01-07 12:07:00,2022-01-07 14:36:00
9,2,03LIC_1034,24.0,0.0,0.0,24.0,,2022-01-07 13:33:00,2022-01-07 13:36:00,3.0,2022-01-07 12:07:00,2022-01-07 14:36:00


In [72]:
# Step 6: Generate and save episode visualizations for all filtered episodes (NO target tag actions)

PLOT_CONTEXT_MINUTES = 60  # matches AlarmEnd + 60 min window
episodes_to_plot = filtered_episodes.copy()

print(f"Generating and saving visualizations for {len(episodes_to_plot)} episodes with NO target tag actions...")
print(f"Output directory: {output_dir}")
print(f"Context minutes: {PLOT_CONTEXT_MINUTES}")
print(f"{'='*70}")

saved_count = 0
error_count = 0

for idx, (_, episode) in enumerate(episodes_to_plot.iterrows()):
    try:
        episode_id = episode['EpisodeID']
        window_start = episode['window_start']
        window_end = episode['window_end']
        
        # Create visualization with timestamps in title
        fig = create_episode_visualization_with_timestamps(
            episode,
            op_pv_data_df,
            combined_pv_events_df,
            context_minutes=PLOT_CONTEXT_MINUTES
        )
        
        # Save as HTML with timestamps in filename
        # Format: episode_XXXX_YYYYMMDD_HHMM_to_YYYYMMDD_HHMM.html
        start_str = window_start.strftime("%Y%m%d_%H%M")
        end_str = window_end.strftime("%Y%m%d_%H%M")
        filename = f"episode_{episode_id:04d}_{start_str}_to_{end_str}.html"
        filepath = output_dir / filename
        fig.write_html(filepath, include_plotlyjs='cdn')
        
        saved_count += 1
        
        # Progress update every 20 episodes
        if (idx + 1) % 20 == 0:
            print(f"   Progress: {idx + 1}/{len(episodes_to_plot)} episodes saved...")
            
    except Exception as e:
        error_count += 1
        print(f"   ⚠️ Error saving episode {episode_id}: {str(e)}")

print(f"\n{'='*70}")
print(f"✅ Saved {saved_count} episode visualizations to: {output_dir}/")
print(f"📊 Excel file: {output_dir}/episodes_no_target_tags_actions.xlsx")
print(f"   Filename format: episode_XXXX_YYYYMMDD_HHMM_to_YYYYMMDD_HHMM.html")
if error_count > 0:
    print(f"⚠️ Errors: {error_count}")

Generating and saving visualizations for 282 episodes with NO target tag actions...
Output directory: RESULTS/episode_no_target_tags_plots
Context minutes: 60
   Progress: 20/282 episodes saved...
   Progress: 40/282 episodes saved...
   Progress: 60/282 episodes saved...
   Progress: 80/282 episodes saved...
   Progress: 100/282 episodes saved...
   Progress: 120/282 episodes saved...
   Progress: 140/282 episodes saved...
   Progress: 160/282 episodes saved...
   Progress: 180/282 episodes saved...
   Progress: 200/282 episodes saved...
   Progress: 220/282 episodes saved...
   Progress: 240/282 episodes saved...
   Progress: 260/282 episodes saved...
   Progress: 280/282 episodes saved...

✅ Saved 282 episode visualizations to: RESULTS/episode_no_target_tags_plots/
📊 Excel file: RESULTS/episode_no_target_tags_plots/episodes_no_target_tags_actions.xlsx
   Filename format: episode_XXXX_YYYYMMDD_HHMM_to_YYYYMMDD_HHMM.html


In [74]:
full_episodes_df = pd.read_excel('/home/h604827/ControlActions/RESULTS/episode_all_operator_action_plots/episodes_all_with_actions_and_deviations.xlsx')
full_episodes_df[(full_episodes_df['OperatedTagsCount'] == 1) & (full_episodes_df['OperatedTags'].str.contains('03FIC_3435'))]

,EpisodeID,AlarmStart,AlarmEnd,AlarmDurationMinutes,TotalWindowMinutes,OperatedTags,OperatedTagsCount,DeviatedTags,DeviatedTagsCount,HasOperatorActions,HasOnlyTargetTags,Has1071Action,Has1016Action,Has1013Action
5,6,2022-01-07 19:56:00,2022-01-07 20:02:00,6,152,03FIC_3435,1,"02FI_1000.PV, 03FIC_1085.PV, 03FIC_3415.PV, 03...",21,True,False,False,False,False
6,7,2022-01-07 23:28:00,2022-01-07 23:31:00,3,149,03FIC_3435,1,"02FI_1000.PV, 03FIC_1085.PV, 03FI_1141A.PV, 03...",22,True,False,False,False,False
7,8,2022-01-10 10:09:00,2022-01-10 10:15:00,6,152,03FIC_3435,1,"02FI_1000.PV, 03FIC_1085.PV, 03FIC_3415.PV, 03...",19,True,False,False,False,False
8,9,2022-01-12 10:41:00,2022-01-12 10:45:00,4,150,03FIC_3435,1,"02FI_1000.PV, 03FIC_1085.PV, 03FI_1141A.PV, 03...",17,True,False,False,False,False
9,10,2022-01-14 11:28:00,2022-01-14 11:35:00,7,153,03FIC_3435,1,"02FI_1000.PV, 03FIC_3415.PV, 03FI_1141A.PV, 03...",13,True,False,False,False,False
10,11,2022-01-16 04:17:00,2022-01-16 04:24:00,7,153,03FIC_3435,1,"03FIC_1085.PV, 03FIC_3415.PV, 03FI_1141A.PV, 0...",22,True,False,False,False,False
12,13,2022-01-22 11:33:00,2022-01-22 11:37:00,4,150,03FIC_3435,1,"02FI_1000.PV, 03FIC_1085.PV, 03FIC_3415.PV, 03...",21,True,False,False,False,False
30,31,2022-02-11 10:15:00,2022-02-11 10:22:00,7,153,03FIC_3435,1,"02FI_1000.PV, 03FIC_1085.PV, 03FIC_3415.PV, 03...",16,True,False,False,False,False
40,41,2022-02-16 07:16:00,2022-02-16 07:24:00,8,154,03FIC_3435,1,"03FIC_3415.PV, 03FI_1141A.PV, 03LIC_1071.PV, 0...",15,True,False,False,False,False
71,72,2022-05-19 01:52:00,2022-05-19 01:58:00,6,152,03FIC_3435,1,"02FI_1000.PV, 03FIC_1085.PV, 03FI_1141A.PV, 03...",16,True,False,False,False,False
